In [1]:
# Cargar librerías
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import mlflow
import mlflow.sklearn


In [2]:
# Leer datos
df = pd.read_csv("data/diabetes.csv")



In [3]:
# identifica las columnas que tiene valores 0
(df==0).sum()

Pregnancies                 111
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                     500
dtype: int64

In [4]:
# Reemplazar ceros en columnas específicas por NaN
cols_to_clean = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin']
# No se consideran algunas, por ejemplo Pregnancies (por qué si es posible que se tengan cero embarazos)

# Reemplazar ceros por NaN para poder tratarlos como datos faltantes
for col in cols_to_clean:
    df[col] = df[col].replace(0, np.nan)

print(df.head())

# Reemplazar NaN con la moda (valor más frecuente) de cada columna
for col in cols_to_clean:
    moda = df[col].mode()[0]
    print(moda)
    df[col] = df[col].fillna(moda)

print(df)

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6    148.0           72.0           35.0      NaN  33.6   
1            1     85.0           66.0           29.0      NaN  26.6   
2            8    183.0           64.0            NaN      NaN  23.3   
3            1     89.0           66.0           23.0     94.0  28.1   
4            0    137.0           40.0           35.0    168.0  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  
99.0
70.0
32.0
105.0
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6    148.0           72.0           35.0    105.0  33.6   
1              1     85.0           66.0           29.0    105.0  26.6   
2              8    183.0           64.0           32.0    1

In [5]:
# Separar datos
X = df.drop("Outcome", axis=1)
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# agregar el traking uri de mlflow
mlflow.set_tracking_uri("http://localhost:9090")

# agregar set_experiment con nombre: ClasificadorDemoDiabetes
mlflow.set_experiment(experiment_name="ClasificadorDemoDiabetes")


<Experiment: artifact_location='mlflow-artifacts:/562461387781550669', creation_time=1747442616555, experiment_id='562461387781550669', last_update_time=1747442616555, lifecycle_stage='active', name='ClasificadorDemoDiabetes', tags={}>

In [7]:
param_variaciones = pd.read_csv("data/logreg_variaciones_educativas.csv")
print(param_variaciones.head())

for row in param_variaciones.iterrows():
    print(row)

        run_id  logreg_C  logreg_max_iter     solver penalty
0  practica_01      0.01              400  liblinear      l2
1  practica_02      1.12              300      lbfgs      l2
2  practica_03      2.23              500  liblinear      l2
3  practica_04      3.34              200  liblinear      l2
4  practica_05      4.45              300  liblinear      l2
(0, run_id             practica_01
logreg_C                  0.01
logreg_max_iter            400
solver               liblinear
penalty                     l2
Name: 0, dtype: object)
(1, run_id             practica_02
logreg_C                  1.12
logreg_max_iter            300
solver                   lbfgs
penalty                     l2
Name: 1, dtype: object)
(2, run_id             practica_03
logreg_C                  2.23
logreg_max_iter            500
solver               liblinear
penalty                     l2
Name: 2, dtype: object)
(3, run_id             practica_04
logreg_C                  3.34
logreg_max_iter    

In [8]:
# Entrenamiento y registro con MLflow
C = 1.0
max_iter = 1000
# usar los parámetros del archivo logreg_variaciones_educativas.csv
# para los parámtros C, max_iter, solver y penalty de LogisticRegresion
# se debe genear un run por cada fila del archivo, usando sus parámetros
# Analizar que usar
param_variaciones = pd.read_csv("data/logreg_variaciones_educativas.csv")
#print(param_variaciones.head())

for i,row in param_variaciones.iterrows():
    print(row['logreg_C'])

    # agregar la línea del start_run
    with mlflow.start_run(run_name=f"ClasificadorDemoDiabetes_{row['run_id']}"):
        # El pipeline usar StandarScaler para que todos los valore numéricos estén en la misma escala
        # no cambiar dicha línea
        # LogisticRegresion deben variar sus parámetros en cada iteración.
        pipeline = Pipeline([
                ("scaler", StandardScaler()),
                ("clf", LogisticRegression(
                    C=row['logreg_C'],
                    max_iter=row['logreg_max_iter'],
                    solver=row['solver'],
                    penalty=row['penalty']
                ))
            ])
        
        # Entrenar y evaluar
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
    
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
    
        # recuerde que log_param y log_metric debe ir guardando los valores
        # dependiendo del run correspondiente de cada fila
        mlflow.log_param("logreg_C", C)
        mlflow.log_param("logreg_max_iter", max_iter)
        mlflow.log_param("solver", "liblinear")
        mlflow.log_param("penalty", "l2")
        
        mlflow.log_metric("accuracy", acc)
        mlflow.log_metric("precision", prec)
        
        # Guardar el pipeline completo
        mlflow.sklearn.log_model(pipeline, "modelo_pipeline")
        
        print(" Modelo registrado en MLflow")
        print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f}")


0.01
 Modelo registrado en MLflow
Accuracy: 0.7208 | Precision: 0.6034
1.12
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
2.23
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
3.34
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
4.45
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
5.5600000000000005
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
6.67
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
7.78
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
8.89
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
10.0
 Modelo registrado en MLflow
Accuracy: 0.7532 | Precision: 0.6667
